In [1]:
from __future__ import print_function, division
import time, os
import numpy as np
import matplotlib.pyplot as plt
import sys
import networkx as nx
import pandas

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [17]:
snapshots_df_full = pandas.read_csv('/home/cs231n/data/Field_Snaps_With_warranty.txt')
selected_snapshots_full = snapshots_df_full[['Veh Ref ID',
 'Event DateTime',
 'Event Type Description',
 'Acc Pedal Position',
 'Ambient Air Temp',
 'Barometric Press',
 'Brake Switch',
 'Bus Utilization',
 'Cat Intake Gas Temp',
 'Cat Outlet Gas Temp',
 'Clutch Switch',
 'Cmd Eng Fuel Press',
 'Cruise Status',
 'Dpf Regen Inhibit Sw',
 'Dpf Thermal Mngmnt',
 'Drvr Demand Torque',
 'Eng Air Flow Rate',
 'Eng Avg Fuel Econ',
 'Eng Coolant Level',
 'Eng Coolant Temp',
 'Eng Demand Torque',
 'Eng DPF Intake Press',
 'Eng Egr Valve Pos',
 'Eng Exhaust Gas Temp',
 'Eng Fuel Del Press',
 'EngFuelTemp1',
 'Engine Speed',
 'Eng Man Abs Pressure',
 'Eng Oil Pressure',
 'EngInjRail1Press',
 'EngIntakeMan1Temp',
 'EngOilTemp1',
 'Eng Percent Torque',
 'EngTurbo1Boost',
 'EngTurbo1Pos',
 'EngTurbo1Speed',
 'Event - All Lamps On Time Hr',
 'Event - Amber Lamp Time Hr',
 'Event - Mil Lamp Time Hr',
 'Event - Red Lamp Time Hr',
 'Exhaust Tank Level',
 'Exhaust Tank Temp',
 'Fan Speed',
 'Keyswitch Bat Pot',
 'Part Trap Diff Press',
 'Part Trap Out Temp',
 'Scr Intake Gas Temp',
 'Scr Outlet Gas Temp',
 'Vehicle Speed',
 'Population',
 'DTCID',
 'Trip Distance',
 'Trip Idle Time',
 'Trip Run Time',
 'Altitude',
 'Engine Start Ambient',
 'Engine Start Coolant',
 'Latitude',
 'Longitude',
 'Lifetime Idle Hours',
 'Lifetime Idle Fuel',
 'Lifetime Fuel',
 'Lifetime Distance',
 'Lifetime Engine Hours']]
selected_repairs_full = pandas.read_csv('/home/cs231n/data/repairs.csv')[[
 'Chassis\nReference\nNumber',
 'Model Vehicle',
 'Build_Dt',
 'Dlvry_Dt',
 'In Service Date',
 'Miles',
 'Rpr_Dt',
 'ATA3',
 'ATA3Desc',
 'ATA6',
 'ATA6Desc',
 'ATA9',
 'ATA9Desc',
 'Fail Type',
 'Repair Cost']]
selected_repairs_full = selected_repairs_full[selected_repairs_full['Chassis\nReference\nNumber'].notnull()] 

In [18]:
selected_repairs = selected_repairs_full[selected_repairs_full['Repair Cost'].isin(['medium', 'high ', 'very high'])]
veh_ids = selected_repairs['Chassis\nReference\nNumber'].unique()
selected_snapshots = selected_snapshots_full[selected_snapshots_full['Veh Ref ID'].isin(veh_ids)]

In [46]:
selected_repairs['ATA9'].unique()

array([ 44003217,  43001071,  45021003,  44003015,  45002074,  44004001,
        44014014,  45002072,  45002000,  45007001,  45007031,  45018000,
        44003016,  43001058,  43004000,  44003027,  45008098,  45008053,
        45013001,  43004072,  43001049,  13009001, 101001001,  43007023,
         3001082,  43006007,  45007007,  43007096])

In [5]:
#---------------------------------------
# Data-Cleaning - selected_snapshots (part 2)
#---------------------------------------

# Impute missing values using mean value, i.e. E(value | vehicle id)
fields = ['Eng Avg Fuel Econ', 'Eng DPF Intake Press', 'Eng Fuel Del Press', 'EngInjRail1Press', 'EngIntakeMan1Temp', 'Fan Speed']
for f in fields:
    noData_vehID = selected_snapshots[selected_snapshots[f].isnull()]['Veh Ref ID'].unique()
    for veh_id in noData_vehID:
        # compute mean value of field f for that vehicle
        value = selected_snapshots[selected_snapshots['Veh Ref ID'] ==  veh_id][f].mean()
        
        #update missing value with mean value
        idx = selected_snapshots.index[selected_snapshots[f].isnull()]
        selected_snapshots.loc[idx,f] = value

nullTable = selected_snapshots.isnull().sum()
nullTable[nullTable > 0]/len(selected_snapshots)

#---------------------------------------
# Data-Cleaning - selected_repairs
#---------------------------------------
#filter away vehicles that broke down before delivery & service (veh 616: accident during undecking) 
selected_repairs = selected_repairs[selected_repairs['Miles']>0]   
selected_repairs.shape   # before: 853x15, after: 852x15

# identify and remove rows with no snapshot data prior to repair date
nodata_veh_repair = []
for veh_id in selected_repairs['Chassis\nReference\nNumber'].unique():
    v_snapshots = selected_snapshots[selected_snapshots['Veh Ref ID'] == veh_id].sort_values(by='Event DateTime')
    repairDate = (selected_repairs[selected_repairs['Chassis\nReference\nNumber']==veh_id]['Rpr_Dt'])    
    for r in repairDate:
        if len(v_snapshots[v_snapshots['Event DateTime']<=r]) == 0: #no snapshot data prior to repair date
            nodata_veh_repair.append({'Chassis\nReference\nNumber':veh_id,'Rpr_Dt':r})
            #remove that row in selected_repair
            idx = selected_repairs.index[(selected_repairs['Chassis\nReference\nNumber'] == veh_id) & (selected_repairs['Rpr_Dt'] == r)]
            selected_repairs = selected_repairs.drop(idx)
            
nodata_veh_repair = pandas.DataFrame(nodata_veh_repair)
selected_repairs.shape  # before: 852x15, after: 713x15

# check for other null entries:
# ('Chasis\nReference\nNumber', 'ATA9', 'Rpr_Dt') has no null entries. Sufficient to link to snapshot data
selected_repairs.isnull().sum()

/home/cs231n/myVE35/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Chassis\nReference\nNumber    0
Model Vehicle                 0
Build_Dt                      0
Dlvry_Dt                      0
In Service Date               0
Miles                         0
Rpr_Dt                        0
ATA3                          0
ATA3Desc                      0
ATA6                          0
ATA6Desc                      0
ATA9                          0
ATA9Desc                      0
Fail Type                     0
Repair Cost                   0
dtype: int64

In [19]:
selected_snapshots.to_csv('/home/cs231n/data/snapshots_cleaned.csv')
selected_repairs.to_csv('/home/cs231n/data/repairs_cleaned.csv')

In [20]:
# 6/14/2016 20:17:07
s_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y %H:%M:%S')
snapshots_df = pandas.read_csv('/home/cs231n/data/snapshots_cleaned.csv', converters={'Event DateTime':s_to_date})

In [21]:
# 2/3/2016
r_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y')
repairs_df = pandas.read_csv('/home/cs231n/data/repairs_cleaned.csv', converters={'Rpr_Dt':r_to_date})

In [22]:
snapshots_df.to_csv('/home/cs231n/data/snapshots_cleaned.csv')
repairs_df.to_csv('/home/cs231n/data/repairs_cleaned.csv')

In [23]:
def get_012_dates(end_date):
    ## split into 0 (10+), 1 (5-10), 2(0-5)
    two_end = end_date
    one_end = pandas.to_datetime(two_end) + pandas.DateOffset(days=-5)
    zero_end = one_end + pandas.DateOffset(days=-5)
    return (zero_end, one_end, two_end)

In [35]:
def get_repair_slices(veh_ids, snapshots, repairs, code='ATA9'):
    repair_slices = {}
    for veh_id in veh_ids:
        v_snapshots = snapshots[snapshots['Veh Ref ID'] == veh_id].sort_values(by='Event DateTime')
        v_repairs = repairs[repairs['Chassis\nReference\nNumber'] == veh_id].sort_values(by='Rpr_Dt')

        start_date = pandas.to_datetime('1/1/2000') ## in past so first snapshot is captured
        event_dt_key = 'Event DateTime'

        repair_slices[veh_id] = {}
        veh_slices = repair_slices[veh_id]

        ## Best indicator of repair type is the ATA9 code
        ## Iterate over each repair type and append slices
        for repair_type, repair_group in v_repairs.groupby([code]):  #ZY: veh_repairs --> v_repairs
            start = start_date
            end = -1

            ## dates of all of the repairs in that group
            repair_dates = repair_group.sort_values(by='Rpr_Dt')['Rpr_Dt']
            r_size = len(repair_dates)

            ## for each repair type, grab slices of snapshots
            veh_slices_repair = {0:[],1:[],2:[]}
            for repair in repair_group.itertuples():
                end = repair[8] ##['Rpr_Dt']

                (zero_end, one_end, two_end) = get_012_dates(end)
                
                # divide up snapshots into 0, 1, 2 slices
                two_mask = (v_snapshots[event_dt_key] > one_end) & (v_snapshots[event_dt_key] <= end)
                one_mask = (v_snapshots[event_dt_key] > zero_end) & (v_snapshots[event_dt_key] <= one_end)
                zero_mask = (v_snapshots[event_dt_key] >= start) & (v_snapshots[event_dt_key] <= zero_end)
                two_slices = v_snapshots.loc[two_mask]
                one_slices = v_snapshots.loc[one_mask]
                zero_slices = v_snapshots.loc[zero_mask]

                if len(two_slices) > 0:
                    veh_slices_repair[2].append(two_slices)
                if len(one_slices) > 0:
                    veh_slices_repair[1].append(one_slices)
                if len(zero_slices) > 0:
                    veh_slices_repair[0].append(zero_slices)

                ## reset start to end for next iteration
                start = end
            
            if len(veh_slices_repair[0]) > 0 or len(veh_slices_repair[1]) > 0 or len(veh_slices_repair[2]) > 0:
                veh_slices[repair_type] = veh_slices_repair

    return repair_slices

In [36]:
repair_slices_all = get_repair_slices(veh_ids, snapshots_df, repairs_df)

In [65]:
repair_slices_all[259][44004001][2][0]

,Unnamed: 0,Veh Ref ID,Event DateTime,Event Type Description,Acc Pedal Position,Ambient Air Temp,Barometric Press,Brake Switch,Bus Utilization,Cat Intake Gas Temp,...,Altitude,Engine Start Ambient,Engine Start Coolant,Latitude,Longitude,Lifetime Idle Hours,Lifetime Idle Fuel,Lifetime Fuel,Lifetime Distance,Lifetime Engine Hours
107713,841368,259.0,2016-05-04 20:58:23,trip_start,0.0,32.68,99.0,0.0,46.0,71.00,...,241.0,32.77,27.77,34.86,-79.1,1.75,1.58,4.87,28.39,3.05
107716,841371,259.0,2016-05-04 21:35:12,trip_end,0.0,32.09,99.5,0.0,53.0,110.37,...,263.0,32.77,27.77,34.86,-79.1,2.40,1.98,5.37,28.49,3.70
107874,841529,259.0,2016-05-05 12:58:36,trip_start,0.0,15.78,98.5,1.0,53.0,58.68,...,278.0,14.99,22.77,34.86,-79.1,2.40,1.98,5.37,28.49,3.70
107873,841528,259.0,2016-05-05 12:58:42,trip_end,0.0,15.78,99.0,0.0,53.0,61.59,...,268.0,14.99,22.77,34.86,-79.1,2.40,1.98,5.37,28.49,3.70
107858,841513,259.0,2016-05-05 13:21:40,trip_start,0.0,17.28,99.0,0.0,47.0,46.78,...,273.0,16.66,25.55,34.86,-79.1,2.40,1.98,5.37,28.49,3.75
107853,841508,259.0,2016-05-05 13:29:29,trip_end,0.0,17.37,99.0,0.0,46.0,74.68,...,278.0,16.66,25.55,34.86,-79.1,2.55,2.11,5.50,28.49,3.85
107861,841516,259.0,2016-05-05 13:51:22,trip_start,0.0,17.09,99.0,1.0,46.0,55.00,...,276.0,16.66,25.55,34.86,-79.1,2.55,2.11,5.50,28.49,3.90
107704,841359,259.0,2016-05-05 13:52:10,trip_end,5.2,17.00,99.0,0.0,51.0,97.00,...,256.0,16.66,25.55,34.86,-79.1,2.55,2.11,5.50,28.49,3.90
107662,841317,259.0,2016-05-05 16:48:57,trip_start,17.2,18.09,99.0,0.0,48.0,111.09,...,260.0,17.77,34.99,34.86,-79.1,2.60,2.11,5.62,28.69,3.95
107843,841498,259.0,2016-05-05 16:49:29,trip_end,0.0,18.18,99.0,0.0,49.0,111.59,...,255.0,17.77,34.99,34.86,-79.1,2.60,2.11,5.62,28.79,3.95


In [56]:
selected_repairs_high = selected_repairs[selected_repairs['Repair Cost'].isin(['medium', 'high ', 'very high'])]
veh_ids = selected_repairs_high['Chassis\nReference\nNumber'].unique()
veh_ids

array([ 208.,  761.,  765.,  158.,  123.,  301.,   60.,  182.,  543.,
        319.,  320.,  616.,   27.,  460.,   18.,  767.,  470.,  270.,
         25.,  651.,  245.,  233.,  558.,  231.,  605.,  429.,  371.,
        317.,  703.,  441.,  165.,  178.,   75.,  251.,  448.,  578.,
        472.,  597.,  504.,  467.,  772.,  184.,  236.,  283.,  735.,
        809.,  724.,  571.,  679.,  673.,  555.,  768.,  218.,  633.,
        340.,  168.,  674.,   73.,  612.,  226.,  376.,  249.,  297.,
        634.,  259.,  409.,  690.,  719.,  584.,  556.,  102.,  192.,
        180.,  495.,  179.,  436.,  291.,  602.,  516.,  515.,  118.,
        211.,  113.,  752.,  424.,  730.,  810.,  808.,  750.,   71.,
        412.,  722.,  453.,  751.,  692.,  163.,  707.,  382.,  408.,
        428.,  137.,  801.,   43.,  120.,    4.,  373.,  725.,  748.,
        791.,  491.,   40.,  797.,  646.,  222.])